In [118]:
import os
import psycopg2

from dotenv import load_dotenv
# Le .env est aussi à la racine
load_dotenv(".env")

USER_PSQL = os.environ.get("POSTGRESQL_LOCAL_USER")
PASSWORD_PSQL = os.environ.get("POSTGRESQL_LOCAL_PASSWORD")

conn = psycopg2.connect(user=USER_PSQL, password=PASSWORD_PSQL,dbname="app_auth", host="localhost", port="5432")
conn.autocommit=True

In [61]:
from psycopg2 import sql
from faker import Faker
import random
from datetime import datetime, timedelta

try:

    cur = conn.cursor()

    # Pour pouvoir ré exécuter le code
    cur.execute("DROP TABLE IF EXISTS user_for_comparison CASCADE;")

    # Création d'une table notebooks
    cur.execute("""
       CREATE TABLE user_for_comparison (
            user_id SERIAL PRIMARY KEY,
            firstname VARCHAR(255),
            lastname VARCHAR(255),
            email VARCHAR(255) UNIQUE,
            username VARCHAR(255) UNIQUE CHECK (LENGTH(password) > 8),
            password VARCHAR(255) CHECK (LENGTH(password) > 8),
            created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
        );
    """)
    print("Table user_for_comparison successfully.")

    fake = Faker()

    # Générer les INSERT INTO pour les utilisateurs
    for i in range(1, 100001):
        user_name = fake.user_name()
        while len(user_name) < 10:
            user_name = f"{fake.user_name()}"

        #Génération d'un user_name unique et email également
        user_name = f"{user_name}{random.randint(1, 100000)}_{random.randint(1, 100000)}"
        email = f"{random.randint(1, 100000)}_{fake.email()}"
        
        cur.execute(f"""
           INSERT INTO user_for_comparison (firstname, lastname, email, username, password, created_at) VALUES ('{fake.first_name()}', '{fake.last_name()}', '{email}', '{user_name}', '{fake.password(length=random.randint(9, 12))}', '{fake.date_time_between(start_date=datetime(2023,1,1), end_date=datetime(2023,6,30))}');
        """)
        
    print("data user_for_comparison successfully.")


    # Fermeture de la connexion
    cur.close()
    conn.close()
except psycopg2.Error as e:
    print(f"Error connecting to PostgreSQL database: {e}")

Table user_for_comparison successfully.
data user_for_comparison successfully.


In [116]:
from psycopg2 import sql
import time
import timeit

try:

    cur = conn.cursor()
    cur.execute("""
        SELECT username 
        FROM user_for_comparison
        WHERE user_id = 258;
    """)
    username1 = str(cur.fetchone())[2:-3]
    cur.execute("""
        SELECT username 
        FROM user_for_comparison
        WHERE user_id = 5245;
    """)
    username2 = str(cur.fetchone())[2:-3]
    cur.execute("""
        SELECT username 
        FROM user_for_comparison
        WHERE user_id = 50555;
    """)
    username3 = str(cur.fetchone())[2:-3]
    cur.execute("""
        SELECT username 
        FROM user_for_comparison
        WHERE user_id = 99999;
    """)
    username4 = str(cur.fetchone())[2:-3]

    print("Question 2 :")
    print(f"Recherche de l'utilisateur avec un username : {username1} ")
    temps = timeit.timeit(lambda: cur.execute(f"""
        SELECT * 
        FROM user_for_comparison
        WHERE username = '{username1}';
    """), number=100)
    print(f"Temps d'éxecution  : {temps} \n")


    print(f"Recherche de l'utilisateur avec un username : {username2}")
    temps = timeit.timeit(lambda: cur.execute(f"""
        SELECT * 
        FROM user_for_comparison
        WHERE username = ' {username2}';
    """), number=100)
    print(f"Temps d'éxecution  : {temps} \n")


    print(f"Recherche de l'utilisateur avec un username : {username3}")
    temps = timeit.timeit(lambda: cur.execute(f"""
        SELECT * 
        FROM user_for_comparison
        WHERE username = '{username3}';
    """), number=100)
    print(f"Temps d'éxecution  : {temps} \n")


    print(f"Recherche de l'utilisateur avec un username : {username4}")
    temps = timeit.timeit(lambda: cur.execute(f"""
        SELECT * 
        FROM user_for_comparison
        WHERE username = '{username4}';
    """), number=100)
    print(f"Temps d'éxecution  : {temps} \n")



    # Fermeture de la connexion
    cur.close()
    conn.close()
except psycopg2.Error as e:
    print(f"Error connecting to PostgreSQL database: {e}")

laurensanders73163_15599
Question 2 :
Recherche de l'utilisateur avec un username : laurensanders73163_15599 
Temps d'éxecution  : 0.008398899997700937 

Recherche de l'utilisateur avec un username : perkinsjames8099_3134
Temps d'éxecution  : 0.006953600008273497 

Recherche de l'utilisateur avec un username : garciatyler43432_40819
Temps d'éxecution  : 0.007232999996631406 

Recherche de l'utilisateur avec un username : grayelizabeth25874_73961
Temps d'éxecution  : 0.0062938000046415254 



In [119]:
from psycopg2 import sql
import time
import timeit

try:

    cur = conn.cursor()

    cur.execute("""
        SELECT username 
        FROM user_for_comparison
        WHERE user_id = 258;
    """)
    username1 = str(cur.fetchone())[2:-3]
    cur.execute("""
        SELECT username 
        FROM user_for_comparison
        WHERE user_id = 5245;
    """)
    username2 = str(cur.fetchone())[2:-3]
    cur.execute("""
        SELECT username 
        FROM user_for_comparison
        WHERE user_id = 50555;
    """)
    username3 = str(cur.fetchone())[2:-3]
    cur.execute("""
        SELECT username 
        FROM user_for_comparison
        WHERE user_id = 99999;
    """)
    username4 = str(cur.fetchone())[2:-3]


    cur.execute("""
        DROP INDEX IF EXISTS index_username
    """)
    cur.execute("""
        CREATE INDEX index_username ON user_for_comparison (username)
    """)
    print("create Index succesfully")

    print("Question 3 :")
    print(f"Recherche de l'utilisateur avec un username : {username1} ")
    temps = timeit.timeit(lambda: cur.execute(f"""
        SELECT * 
        FROM user_for_comparison
        WHERE username = '{username1}';
    """), number=1)
    print(f"Temps d'éxecution  : {temps} \n")


    print(f"Recherche de l'utilisateur avec un username : {username2}")
    temps = timeit.timeit(lambda: cur.execute(f"""
        SELECT * 
        FROM user_for_comparison
        WHERE username = '{username2}';
    """), number=1)
    print(f"Temps d'éxecution  : {temps} \n")


    print(f"Recherche de l'utilisateur avec un username : {username3}")
    temps = timeit.timeit(lambda: cur.execute(f"""
        SELECT * 
        FROM user_for_comparison
        WHERE username = '{username3}';
    """), number=1)
    print(f"Temps d'éxecution  : {temps} \n")


    print(f"Recherche de l'utilisateur avec un username : {username4}")
    temps = timeit.timeit(lambda: cur.execute(f"""
        SELECT * 
        FROM user_for_comparison
        WHERE username = '{username4}';
    """), number=1)
    print(f"Temps d'éxecution  : {temps} \n")



    # Fermeture de la connexion
    cur.close()
    conn.close()
except psycopg2.Error as e:
    print(f"Error connecting to PostgreSQL database: {e}")

create Index succesfully
Question 3 :
Recherche de l'utilisateur avec un username : laurensanders73163_15599 
Temps d'éxecution  : 0.0009516999998595566 

Recherche de l'utilisateur avec un username : perkinsjames8099_3134
Temps d'éxecution  : 0.00019690000044647604 

Recherche de l'utilisateur avec un username : garciatyler43432_40819
Temps d'éxecution  : 0.00015350000467151403 

Recherche de l'utilisateur avec un username : grayelizabeth25874_73961
Temps d'éxecution  : 9.790000331122428e-05 

